# Building Value-driven Dashboards in Python

In this notebook, we will go through the process of creating a dashboard written in Python, for exploring the City of Melbourne's [Pedestrian Traffic Hourly Counts Dataset](https://data.melbourne.vic.gov.au/Transport/Pedestrian-Counting-System-2009-to-Present-counts-/b2ak-trbp).

We will primarily be using the following Python libraries/tools:
* [JupyterLab](https://jupyterlab.readthedocs.io/): An interactive development environment that is well suited for exploratory data analysis. 
* [Dash](https://plotly.com/dash/): a framework for building analytic web-apps in Python.
* [Pandas](https://pandas.pydata.org): a library for analysing and processing tabular data.

See the instructions in the README at the base of this repository on how to setup the environment you'll need for working through this notebook.

_Note:_ You should also be able to use the original Jupyter Notebook (as opposed to JupyterLab) to complete this workshop, however I would recommend using JupyterLab, as this is what I used while developing the dashboard, and more generally because it offers a superior experience.

## Goals and Motivation

The goals of this notebook are as follows:

* Understand the types of options available when constructing dashboards and the kinds of contexts when choosing Python might be a good choice.
* Become familiar with the steps involved in the end-to-end process of producing a Python-based dashboard
* Help you become familiar with components of a tech stack that is well suited for this task.
* Answer some questions about pedestrian traffic patterns in Melbourne's CBD. 

_Note:_ There is not one single best-practice methodology for developing dashboards in Python. This is my attempt to distil some useful strategies and processes from what I've learnt from my experiences. 

## The Dataset
The dataset we'll be using is the Melbourne City Council's Pedestrian Counting System dataset, which is part of the council's [Open Data Portal](https://data.melbourne.vic.gov.au):

_This dataset contains hourly pedestrian counts since 2009 from pedestrian sensor devices located across the city. The data is updated on a monthly basis and can be used to determine variations in pedestrian activity throughout the day._

The data that we will be using comes from two separate datasets:

1. [The Pedestrian Counting System dataset](https://data.melbourne.vic.gov.au/Transport/Pedestrian-Counting-System-2009-to-Present-counts-/b2ak-trbp), which contains the hourly traffic data.
2. [Pedestrian Sensor Locations](https://data.melbourne.vic.gov.au/Transport/Pedestrian-Counting-System-Sensor-Locations/h57g-5234) dataset, which contains data about the sensors collecting the above data. 

In [9]:
from pathlib import Path
import pandas as pd

# change this if needed
data_path = Path("..") / "data"

sensor_csv_path = data_path / "Pedestrian_Counting_System_-_Sensor_Locations.csv"
counts_csv_path = data_path / "Pedestrian_Counting_System___2009_to_Present__counts_per_hour_.csv"

sensors_df = pd.read_csv(sensor_csv_path)
counts_df = pd.read_csv(counts_csv_path)

The sensor dataset contains a range of information regarding each sensor. We'll just be using the geographical cordinates of each sensor.

In [12]:
sensors_df.head()

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
0,59,Building 80 RMIT,RMIT_T,2019/02/13,A,NaN,North,South,-37.808256,144.963049,"(-37.80825648, 144.96304859)"
1,23,Spencer St-Collins St (South),Col623_T,2013/09/02,A,NaN,East,West,-37.819093,144.954527,"(-37.81909256, 144.95452749)"
2,20,Chinatown-Lt Bourke St (South),LtB170_T,2013/09/06,A,NaN,East,West,-37.811729,144.968247,"(-37.81172913, 144.9682466)"
3,34,Flinders St-Spark La,Fli32_T,2014/06/08,A,NaN,East,West,-37.815380,144.974150,"(-37.81537985, 144.9741505)"
4,57,Bourke St Bridge,BouBri_T,2018/08/13,A,NaN,West,East,-37.817673,144.950256,"(-37.8176735, 144.95025595)"


In [13]:
counts_df.head()

,ID,Date_Time,Year,Month,Mdate,Day,Time,Sensor_ID,Sensor_Name,Hourly_Counts
0,2887628,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,34,Flinders St-Spark La,300
1,2887629,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,39,Alfred Place,604
2,2887630,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,37,Lygon St (East),216
3,2887631,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,40,Lonsdale St-Spring St (West),627
4,2887632,11/01/2019 05:00:00 PM,2019,November,1,Friday,17,36,Queen St (West),774


## Your Mission

You will assume the role of a data specialist whose has been tasked with the responsibility of developing a dashboard for the non-profit organisation TODO that will assist their analysts understand pedestrian traffic flow around the CBD.

Through the rest of the notebook, you will go on the journey of dashboard creation, from discovery all the way through to deploying the dashboard for analysts to use.


## Discovery

In order to scope out the requirements of this projects, you spent a week talking and having workshops with various stakeholders, including the team of analysts who will use the dashboard and the the team that manages the technical infrastructure. Your findings are as follows:

* The team of analysts that will use dashboard currently use a combination of Python, SQL, and Excel for data wrangling and analysis. 
* For reporting and visualisation they use Tablau but have found this to be limiting.
* For a different dataset, they had previously engaged a delivery team to build a polished React dashboard using D3.js, but were then unable to extend the app when needed enhancements were identified.
* Ideally, they want a solution that they can maintain and extend themselves
* The team is prepared to undergo any training that would be required
* The infrastructure team has recently started using AWS and will be able to provide access to an account. 

You also identify the following information needs that the dashboard should support:
* How does the pedestrian traffic differ across locations and at different times?
* What is peak and off-peak period time for different months and season?
* What is the effect of specific events (eg Australian Open, Moomba, White Night, etc) on pedestrian movement?

## Solution Design

Based on the information gathered during discovery, you propose the following solution:

* The dashboard will be implemented using Dash, with visualisations being produced by the Plotly library.
* The app will be hosted on the org's AWS account.
* You will develop an initial version of the dashboard that features a set of customised and visualisations
* You will provide training to the analytics team on how to modify and extend the dashboard


## Phase 1: Exploratory Data Analysis

The first thing you do is get to understand the data. Not just at a high level, but really dive in and understant it. This is important for:
* identifying data quality or integrity issues
* understanding what kind of applications the data does and does not support

Broadly speaking, this process of exploratory data analysis (EDA) involves two main facets: 
1. Extractiong summary stastistics about data
2. Visualise your data

### Navigating Jupyer

Your Jupyter Notebook is a list of cells that are of two main types:
1. Mardown content (like this cell)
2. Code content (cells with Python in this notebook)

When using the notebook, you are in one of two modes, that function as what you'd imagine from their names:
1. Navigation mode
2. Edit mode

You can use your mouse to navitgate cells (double click activates edit mode) and perform cell operations using the Jupyter menu bar, but the less you need to use the mouse, the more fun you'll have :)

#### Convenient keyboard shortcuts you should familiarise yourself with:

__Navigation mode__
* `up`, `down`: navigate between cells
* `enter`: activate edit mode for the current cell
* `a`: create a new code cell above the current one
* `b`: create a new code cell below the current one
* `m`: change the current cell to be a markdown cell
* `y`: change the current cell to be a code cell
* `d` `d`: Delete the selected cell(s)
* `c`: copy the selected cell(s)
* `v`: paste the copied cell(s)
* `x`: cut the selected cell(s)
* `z`: undo cell operation

__Edit mode__
* `escape`: Switch to navigation mode 

__Edit mode & Navigation mode__
* `control`+`enter`: evaluate the current cell
* `shift`+`enter`: evaluate the current cell and move focus to cell below in navigation mode
* `alt`+`enter`: evaluate the current cell and create a new cell below in edit mode


### A Crash Course in Pandas

Pandas provides you with a powerful interface to help you manipulate and analyse data. It assists you perform the kind of tasks that you might already have used Excel or SQL to perform.

When working with data in Pandas, you typically work with a `DataFrame`, which is a data structure made of of columns and rows. Each column is a `Series`, which is a data structure that contains a sequence of values. If you've used Numpy before, a `Series` is essentially a Numpy array where each element has an index. This index means we can combine different data sources that share the same index, in exactly the same way as merging tables in SQL.

Let's look at the sensor counts `DataFrame` we have already read in above from the CSV.

In [20]:
type(counts_df)

pandas.core.frame.DataFrame

In [28]:
counts_df.tail(100)

,ID,Date_Time,Year,Month,Mdate,Day,Time,Sensor_ID,Sensor_Name,Hourly_Counts
3218358,3218359,06/30/2020 10:00:00 PM,2020,June,30,Tuesday,22,14,Sandridge Bridge,134
3218359,3218360,06/30/2020 10:00:00 PM,2020,June,30,Tuesday,22,26,QV Market-Elizabeth St (West),100
3218360,3218361,06/30/2020 10:00:00 PM,2020,June,30,Tuesday,22,24,Spencer St-Collins St (North),154
3218361,3218362,06/30/2020 10:00:00 PM,2020,June,30,Tuesday,22,23,Spencer St-Collins St (South),38
3218362,3218363,06/30/2020 10:00:00 PM,2020,June,30,Tuesday,22,25,Melbourne Convention Exhibition Centre,71
...,...,...,...,...,...,...,...,...,...,...
3218453,3218454,06/30/2020 11:00:00 PM,2020,June,30,Tuesday,23,62,La Trobe St (North),25
3218454,3218455,06/30/2020 11:00:00 PM,2020,June,30,Tuesday,23,63,231 Bourke St,69
3218455,3218456,06/30/2020 11:00:00 PM,2020,June,30,Tuesday,23,64,Royal Pde-Grattan St,7
3218456,3218457,06/30/2020 11:00:00 PM,2020,June,30,Tuesday,23,65,Swanston St - City Square,32


In [22]:
counts_df.columns

Index(['ID', 'Date_Time', 'Year', 'Month', 'Mdate', 'Day', 'Time', 'Sensor_ID',
       'Sensor_Name', 'Hourly_Counts'],
      dtype='object')

In [25]:
sensors = counts_df["Sensor_Name"]
sensors

0                  Flinders St-Spark La
1                          Alfred Place
2                       Lygon St (East)
3          Lonsdale St-Spring St (West)
4                       Queen St (West)
                       ...             
3218453             La Trobe St (North)
3218454                   231 Bourke St
3218455            Royal Pde-Grattan St
3218456       Swanston St - City Square
3218457               State Library-New
Name: Sensor_Name, Length: 3218458, dtype: object

In [26]:
type(sensors)

pandas.core.series.Series

### Cleaning Data

After inspecting the data, you see that only thing you need to do is make sure that Pandas reads in the `Date_Time` column as a data type suitable for handling date/time, as it isn't much use to us a string (which Pandas actually defaults to loading as an `object`).

This dataset is already pretty clean :)

In [29]:
counts_df.dtypes

ID                int64
Date_Time        object
Year              int64
Month            object
Mdate             int64
Day              object
Time              int64
Sensor_ID         int64
Sensor_Name      object
Hourly_Counts     int64
dtype: object

In [31]:
counts_df = pd.read_csv(counts_csv_path, parse_dates=["Date_Time"])

In [32]:
counts_df.dtypes

ID                        int64
Date_Time        datetime64[ns]
Year                      int64
Month                    object
Mdate                     int64
Day                      object
Time                      int64
Sensor_ID                 int64
Sensor_Name              object
Hourly_Counts             int64
dtype: object

### Exploring our Data

In [33]:
counts_df.shape

(3218458, 10)

In [37]:
# How many years does the data set cover?
years = counts_df["Year"].unique()
sorted(years)

[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [40]:
# how many sensors do we have readings for
sensors = counts_df["Sensor_Name"].unique()
len(sensors)

69

Q: What was the busiest/least busiest month in 2019?

Approach:
1. Filter data to 2019
2. Group counts by month, summing up each month
3. Sort monthly counts

In [42]:
# 1. filtering
filter_mask = counts_df["Year"] == 2019
filter_mask

0           True
1           True
2           True
3           True
4           True
           ...  
3218453    False
3218454    False
3218455    False
3218456    False
3218457    False
Name: Year, Length: 3218458, dtype: bool

In [55]:
# 1. filtering
counts_2019_df = counts_df[filter_mask]
counts_2019_df.head()

,ID,Date_Time,Year,Month,Mdate,Day,Time,Sensor_ID,Sensor_Name,Hourly_Counts
0,2887628,2019-11-01 17:00:00,2019,November,1,Friday,17,34,Flinders St-Spark La,300
1,2887629,2019-11-01 17:00:00,2019,November,1,Friday,17,39,Alfred Place,604
2,2887630,2019-11-01 17:00:00,2019,November,1,Friday,17,37,Lygon St (East),216
3,2887631,2019-11-01 17:00:00,2019,November,1,Friday,17,40,Lonsdale St-Spring St (West),627
4,2887632,2019-11-01 17:00:00,2019,November,1,Friday,17,36,Queen St (West),774


In [52]:
# grouping
months_2019 = counts_df.groupby("Month")["Hourly_Counts"].sum()
months_2019

Month
March        172436211
December     170223773
October      161901572
November     160487798
January      157435313
February     156843588
May          156137355
July         155313182
August       153766468
September    153304281
April        151564562
June         149554005
Name: Hourly_Counts, dtype: int64

In [54]:
months_2019 = months_2019.sort_values(ascending=False)
months_2019

Month
March        172436211
December     170223773
October      161901572
November     160487798
January      157435313
February     156843588
May          156137355
July         155313182
August       153766468
September    153304281
April        151564562
June         149554005
Name: Hourly_Counts, dtype: int64

### Visual Exploration with Plotly

An even better way to answer questions like the last 
Viewing sorted lists of values is not the best way to gather insights from data. 

In [41]:
year_counts_df = counts_df.groupby("Year")["Hourly_Counts"].sum().reset_index()
px.line(year_counts_df, x="Year", y="Hourly_Counts", title="Total Footfalls by Year").update_traces(mode='lines+markers')

NameError: name 'px' is not defined

## Phase 2: Visualisation Development

### Plot 1: Visualising Traffic Size by Sensor

### Plot 2: Visualising Traffic Size by Month

### Plot 3: Visualising Traffic Geographically

### Plot 4: Visualising Traffic Temporally

## Phase 3: Escaping the Notebook

1. capturing repeated actions with abstractions
 * loading and saving data
 * filter data
 * make custom plots from filtered data
2. converting code into a package

Why:
1. gives you tools for performing later analysis faster and will enable cleaner, more maintianable, and more extensable dashboard code.  

## Phase 4: Making the Dashboard

## Phase 5: Deploying the Dashboard